In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import math
import shutil
from  shutil import copyfile


# HANDLE DATA

In [ ]:
TRAIN_PATH = 'D:\\Documents\\GMU\\Clases\\CS747\\Assigments\\HW2\\code\\workData\\train\\'
VAL_PATH='D:\\Documents\\GMU\\Clases\\CS747\\Assigments\\HW2\\code\\workData\\validate\\'
TEST_PATH='D:\\Documents\\GMU\\Clases\\CS747\\Assigments\\HW2\\code\\workData\\test\\'

DATA_PATH = 'D:\\Documents\\GMU\\Clases\\CS747\\Assigments\\HW2\\code\\Data\\'

batch_size = 15 #128
epochs = 75
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [ ]:
os.mkdir('D:\\Documents\\GMU\\Clases\\CS747\\Assigments\\HW2\\code\\workData\\')
os.mkdir(TRAIN_PATH)
os.mkdir(VAL_PATH)
os.mkdir(TEST_PATH)

train_food = os.path.join(TRAIN_PATH, 'food')
train_no_food = os.path.join(TRAIN_PATH, 'no_food')

val_food = os.path.join(VAL_PATH, 'food')
val_no_food = os.path.join(VAL_PATH, 'no_food')

test_food = os.path.join(TEST_PATH, 'food')
test_no_food = os.path.join(TEST_PATH, 'no_food')

os.mkdir(train_food)
os.mkdir(train_no_food)

os.mkdir(val_food)
os.mkdir(val_no_food)

os.mkdir(test_food)
os.mkdir(test_no_food)


In [ ]:
train_food_list = []
train_no_food_list = []

for file in os.listdir(DATA_PATH + 'training\\'):
    if file[:1] == '0':
        train_no_food_list.append(file)
    elif file[:1] == '1':
        train_food_list.append(file)
        
validate_food_list = []
validate_no_food_list = []

for file in os.listdir(DATA_PATH + 'validation\\'):
    if file[:1] == '0':
        validate_no_food_list.append(file)
    elif file[:1] == '1':
        validate_food_list.append(file)
        
test_food_list = []
test_no_food_list = []

for file in os.listdir(DATA_PATH + 'evaluation\\'):
    if file[:1] == '0':
        test_no_food_list.append(file)
    elif file[:1] == '1':
        test_food_list.append(file)

        
print('Images to train: food ['+str(len(train_food_list))+'] no food ['+str(len(train_no_food_list))+']')
print('Images to validate: food ['+str(len(validate_food_list))+'] no food ['+str(len(validate_no_food_list))+']')
print('Images to test: food ['+str(len(test_food_list))+'] no food ['+str(len(test_no_food_list))+']')


stepPerEpoch = math.ceil((len(train_food_list) + len(train_no_food_list)) / batch_size)

print('step per epoch ' + str(stepPerEpoch))

In [ ]:
for file in train_food_list:
    copyfile(os.path.join(DATA_PATH + 'training\\', file), os.path.join(train_food, file))
for file in train_no_food_list:
    copyfile(os.path.join(DATA_PATH + 'training\\', file), os.path.join(train_no_food, file))
    
for file in validate_food_list:
    copyfile(os.path.join(DATA_PATH + 'validation\\', file), os.path.join(val_food, file))
for file in validate_no_food_list:
    copyfile(os.path.join(DATA_PATH + 'validation\\', file), os.path.join(val_no_food, file))
    
for file in test_food_list:
    copyfile(os.path.join(DATA_PATH + 'evaluation\\', file), os.path.join(test_food, file))
for file in train_no_food_list:
    copyfile(os.path.join(DATA_PATH + 'evaluation\\', file), os.path.join(test_no_food, file))
        

# Handle manual CNN

In [ ]:
train_image_generator_no_aumentation = ImageDataGenerator(rescale=0)
validation_image_generator = ImageDataGenerator(rescale=0)

In [ ]:
train_data_gen_no_aumentation = train_image_generator_no_aumentation.flow_from_directory(batch_size=batch_size,
                                                     directory=TRAIN_PATH,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VAL_PATH,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')
test_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=TEST_PATH,
                                                              shuffle= False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
dropout = 0.3

model = Sequential([
    Conv2D(8, 3, padding='same', activation='relu', 
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(dropout),
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    #Conv2D(32, 3, padding='same', activation='relu'),
    #MaxPooling2D(),
    Dropout(dropout),
    Flatten(), # flat the dimension to fit on dense layers
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

history = model.fit(
    train_data_gen_no_aumentation,
    steps_per_epoch=stepPerEpoch,
    epochs=75,
    validation_data=val_data_gen,
    callbacks=[es]
)

In [ ]:
model.summary()

In [ ]:
plot(history)

In [ ]:
test_loss, test_acc = model.evaluate(test_data_gen
                                     , verbose=2)

# VGG16 without augmentation

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
train_image_generator_no_aumentation = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_data_gen_no_aumentation = train_image_generator_no_aumentation.flow_from_directory(batch_size=batch_size,
                                                     directory=TRAIN_PATH,
                                                     shuffle=False,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VAL_PATH,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')
test_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=TEST_PATH,
                                                              shuffle= False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

VGG16_model = VGG16(input_shape = IMG_SHAPE,
                   include_top=False,
                   weights = 'imagenet'
                   )
for layer in VGG16_model.layers:
    layer.trainable = False

model = tf.keras.Sequential([
    VGG16_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [ ]:
VGG16_model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 5)

history = model.fit(
    train_data_gen_no_aumentation,
    steps_per_epoch=stepPerEpoch,
    epochs=75,
    validation_data=val_data_gen,
    callbacks=[es]
)

In [ ]:
model.summary()

In [ ]:
plot(history)

In [ ]:
test_loss, test_acc = model.evaluate(test_data_gen
                                     , verbose=2)

# VGG16 with augmentation

In [ ]:
train_image_generator = ImageDataGenerator(
                                                preprocessing_function=preprocess_input,
                                                rescale=1./255,
                                                rotation_range=45,
                                                width_shift_range=.15,
                                                height_shift_range=.15,
                                                horizontal_flip=True,
                                                vertical_flip = True,
                                                zoom_range=0.3                         
                                            ) # Generator for our training data

validation_image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                     directory=TRAIN_PATH,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VAL_PATH,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')
test_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=TEST_PATH,
                                                              shuffle= False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

VGG16_model = VGG16(input_shape = IMG_SHAPE,
                   include_top=False,
                   weights = 'imagenet'
                   )
for layer in VGG16_model.layers:
    layer.trainable = False

model = tf.keras.Sequential([
    VGG16_model,
    Flatten(),
    Dense(1, activation='sigmoid')
])

In [ ]:
VGG16_model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 10)

history = model.fit(
    train_data_gen,
    steps_per_epoch=stepPerEpoch,
    epochs=75,
    validation_data=val_data_gen,
    callbacks=[es]
)

In [ ]:
plot(history)

In [ ]:
test_loss, test_acc = model.evaluate(test_data_gen
                                     , verbose=2)

# Space for plot functions

In [ ]:
import matplotlib.pyplot as plt

def plot(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
# Print a bar chart with groups

import numpy as np
import matplotlib.pyplot as plt

# set height of bar
# length of these lists determine the number
# of groups (they must all be the same length)
bars1 = [0.9660, 1.0000, 0.9503]
bars2 = [0.8410, 0.9810, 0.9700]

# set width of bar. To work and supply some padding
# the number of groups times barWidth must be
# a little less than 1 (since the next group
# will start at 1, then 2, etc).

barWidth = 0.25
# Set position of bar on X axis
r1 = np.arange(len(bars1))
r2 = [x + barWidth for x in r1]

# Make the plot
plt.bar(r1, bars1, color='green', width=barWidth, edgecolor='white', label='Train')
plt.bar(r2, bars2, color='red', width=barWidth, edgecolor='white', label='Validate')

# Add xticks on the middle of the group bars
plt.xlabel('Different accuracies', fontweight='bold')
plt.xticks([r + barWidth for r in range(len(bars1))], ['CNN of choice', 'VGG16', 'VGG16 with augmentation'])

# Create legend & Show graphic
plt.legend()
plt.show()
#plt.savefig("barChart.pdf",dpi=400,bbox_inches='tight',pad_inches=0.05) # save as a pdf